In [1]:
# Importando a biblioteca necessária
import pandas as pd

# Carregar o dataset com delimitador e codificação corretos
dataset_producao = pd.read_csv(
    'DataSet.dsv', 
    delimiter=';', 
    encoding='latin1', 
    dtype={'CO_PA_AUTORIZ': str, 
           'CO_PA_CNSPAC': str,
           'CO_PA_PROC_ID': str,
           'CO_PA_MVM': str,
           'CO_PA_CMP': str,
           'NU_PA_QTDAPR': str,
           'NU_PA_VALAPR': str
          },
    low_memory=False
)

dataset_producao['NU_PA_QTDAPR'] = dataset_producao['NU_PA_QTDAPR'].str.replace(',', '.').astype(float)
dataset_producao['NU_PA_VALAPR'] = dataset_producao['NU_PA_VALAPR'].str.replace(',', '.').astype(float)

print("===1. Importados os campos CO_PA_AUTORIZ,CO_PA_CNSPAC,CO_PA_PROC_ID,CO_PA_MVM,CO_PA_CMP,NU_PA_QTDAPR e NU_PA_VALAPR do dataset DataSet.dsv")

# 1. Quantidade total e percentual de registros para pacientes identificados e não identificados (CO_PA_CNSPAC)
total_registros = len(dataset_producao)
print(f"Quantidade de registros: {total_registros:,}".replace(',', '.'))
cnspac_identificado = dataset_producao['CO_PA_CNSPAC'].notna().sum()
cnspac_nao_identificado = total_registros - cnspac_identificado
percentual_identificado = (cnspac_identificado / total_registros) * 100
percentual_nao_identificado = (cnspac_nao_identificado / total_registros) * 100

print("\n2. Quantidade e percentual de registros identificados e não identificados:")
print(f"    Pacientes identificados (CO_PA_CNSPAC não nulo): {cnspac_identificado} registros ({percentual_identificado:.2f}%)")
print(f"    Pacientes não identificados (CO_PA_CNSPAC nulo): {cnspac_nao_identificado} registros ({percentual_nao_identificado:.2f}%)")

print("\nRealizando operação de group by ")

dataset_producao_agrupado = dataset_producao.groupby(
    ["CO_PA_AUTORIZ", "CO_PA_CNSPAC", "CO_PA_PROC_ID", "CO_PA_MVM", "CO_PA_CMP"],
    as_index=False                    
).agg({
    'NU_PA_QTDAPR': 'sum',  
    'NU_PA_VALAPR': 'sum'   
})

total_registros_apos_agrupamento=len(dataset_producao_agrupado)
print(f"\n=== Quantidade de registros após agrupamento: {total_registros_apos_agrupamento:,}".replace(',', '.'))

cns_qtd_proced = dataset_producao_agrupado.groupby('CO_PA_CNSPAC').agg(
    contagem_CO_PA_PROC_ID=('CO_PA_PROC_ID', 'count')
).reset_index()

cns_qtd_proced.to_excel('cns_qtd_proced.xlsx', index=False)

print("\nExportado o Dataset com todos os CNS e quantidade de procedimentos de cada CNS (cns_qtd_proced.xlsx) ")

print("\nQuantidade de registros exportados: ", len(cns_qtd_proced))

# Criar datasets com cns cuja contagem de CO_PA_PROC_ID seja maior que 1 e igual a 1

# Criar o dataset secundário onde contagem_CO_PA_PROC_ID > 1
cns_qtd_proced_maior_1 = cns_qtd_proced[cns_qtd_proced['contagem_CO_PA_PROC_ID'] > 1]

# Exibir a quantidade de registros no dataset secundário
print(f"\nQuantidade de registros onde contagem_CO_PA_PROC_ID > 1: {len(cns_qtd_proced_maior_1)}")

# Criar o dataset secundário onde contagem_CO_PA_PROC_ID > 1
cns_qtd_proced_igual_1 = cns_qtd_proced[cns_qtd_proced['contagem_CO_PA_PROC_ID'] == 1]

# Exibir a quantidade de registros no dataset secundário
print(f"\nQuantidade de registros onde contagem_CO_PA_PROC_ID = 1: {len(cns_qtd_proced_igual_1)}")

cns_qtd_proced_maior_1.to_excel("cns_qtd_proced_maior_1.xlsx", index=False)

cns_qtd_proced_igual_1.to_excel("cns_qtd_proced_igual_1.xlsx", index=False)

print(
    "Exportado Datasets: cns_qtd_proced_maior_1.xlsx", len(cns_qtd_proced_maior_1), 
    "registros e cns_qtd_proced_igual_1.xlsx", len(cns_qtd_proced_igual_1), 
    "registros."
)

# Filtrar no dataset_producao_agrupado os CO_PA_CNSPAC não nulos que constam em cns_qtd_proced_maior_1
dataset_filtrado_maior_1 = dataset_producao_agrupado[
    dataset_producao_agrupado['CO_PA_CNSPAC'].notna() & 
    dataset_producao_agrupado['CO_PA_CNSPAC'].str.len() >= 15 &
    dataset_producao_agrupado['CO_PA_CNSPAC'].isin(cns_qtd_proced_maior_1['CO_PA_CNSPAC'])
]

print(f"\nTotal de registros após filtrar CO_PA_CNSPAC com mais de 1 procedimento e não nulo: {len(dataset_filtrado_maior_1)}")

# Exportar o resultado filtrado para Excel (opcional)
dataset_filtrado_maior_1.to_excel('dataset_filtrado_maior_1.xlsx', index=False)

print("\nArquivo 'dataset_filtrado_maior_1.xlsx' exportado com sucesso!")

# Criar a tabela dinâmica
tabela_dinamica = dataset_filtrado_maior_1.pivot_table(
    index='CO_PA_CNSPAC',          # Linhas: CO_PA_CNSPAC
    columns='CO_PA_PROC_ID',       # Colunas: CO_PA_PROC_ID
    values='NU_PA_QTDAPR',         # Valores: NU_PA_QTDAPR
    aggfunc='sum',                 # Agregar com soma
    fill_value=0                   # Preencher valores faltantes com 0
)

# Caso queira exportar para Excel
tabela_dinamica.to_excel('tabela_dinamica.xlsx', index=True)

print("\n FIM.")



===1. Importados os campos CO_PA_AUTORIZ,CO_PA_CNSPAC,CO_PA_PROC_ID,CO_PA_MVM,CO_PA_CMP,NU_PA_QTDAPR e NU_PA_VALAPR do dataset DataSet.dsv
Quantidade de registros: 165.166

2. Quantidade e percentual de registros identificados e não identificados:
    Pacientes identificados (CO_PA_CNSPAC não nulo): 163106 registros (98.75%)
    Pacientes não identificados (CO_PA_CNSPAC nulo): 2060 registros (1.25%)

Realizando operação de group by 

=== Quantidade de registros após agrupamento: 150.519

Exportado o Dataset com todos os CNS e quantidade de procedimentos de cada CNS (cns_qtd_proced.xlsx) 

Quantidade de registros exportados:  107780

Quantidade de registros onde contagem_CO_PA_PROC_ID > 1: 30547

Quantidade de registros onde contagem_CO_PA_PROC_ID = 1: 77233
Exportado Datasets: cns_qtd_proced_maior_1.xlsx 30547 registros e cns_qtd_proced_igual_1.xlsx 77233 registros.

Total de registros após filtrar CO_PA_CNSPAC com mais de 1 procedimento e não nulo: 150519

Arquivo 'dataset_filtrado_ma